In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy
%matplotlib qt
# 数据载入与预处理

df = np.loadtxt('C:/Users/DELL/Desktop/eos/data.txt') # V, T, P ,E

df_E = df[:,-1]
df_V = df[:,0]
df_T = df[:,1]
 
dataX = df[:,0:-1].copy()
lnT = np.log(dataX[:,1])
INCLUDING_LNT = True
if INCLUDING_LNT:
    dataX = np.hstack((dataX[:,:-1], lnT.reshape(-1,1), dataX[:,-1:]))

Data_min = np.min(dataX, axis=0)
Data_max = np.max(dataX, axis=0)
Data_scale = Data_max - Data_min

Vmin, Tmin, Pmin = Data_min[0],Data_min[1],Data_min[-1]
Vmax, Tmax, Pmax = Data_max[0],Data_max[1],Data_max[-1]
Vscale, Tscale, Pscale = Data_scale[0],Data_scale[1],Data_scale[-1]

dataX = (dataX - np.min(dataX, axis=0, keepdims=True))/(np.max(dataX, axis=0, keepdims=True) - np.min(dataX, axis=0, keepdims=True))
X = dataX[:,:-1]
y = dataX[:,-1:]

In [2]:
# 用多项式回归得到P
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, BayesianRidge, ARDRegression

def poly_regression(x_train, y_train, degree, reg_model=LinearRegression()):
    poly = PolynomialFeatures(degree)
    poly_train = poly.fit_transform(x_train)
    reg_p = reg_model.fit(poly_train, y_train.ravel())   
    return reg_p, poly

def make_prediction(reg_model, poly, xtest):
    test_features = poly.transform(xtest)
    p_pred = reg_model.predict(test_features)
    p_pred = p_pred.reshape(p_pred.shape[0],1)
    return p_pred

degree = 4
# model = BayesianRidge()
model = ARDRegression()
# model = LinearRegression()
# model = Lasso(alpha=1e-5, max_iter=1000, warm_start=True, tol=1e-4, selection='random')
# model = Ridge(alpha=0.1, max_iter=1000, tol=1e-4, solver='auto')
# model = ElasticNet(alpha=1e-6, l1_ratio=0.5, max_iter=1000, tol=1e-4, selection='cyclic')

reg_model, poly_feature_trans = poly_regression(X, y, degree, model)


In [3]:
# 定义P(v,t)
def P(v,t):
    v_std = (v-Vmin)/Vscale
    t_std = (t-Tmin)/Tscale
    lnt_std = (np.log(t) - np.min(lnT))/(np.max(lnT) - np.min(lnT))
    p_std = make_prediction(reg_model, poly_feature_trans, np.array([[v_std,t_std,lnt_std]]))[0][0]
    p = p_std*Pscale + Pmin
    return p

In [4]:
# 对P求变上限积分（任意值）得到F
from scipy.integrate import quad

# 积分常数项f(T)
def f(t,zeta):
    f_t = zeta[0] + zeta[1]*t + zeta[2]*(t**zeta[3])
    return f_t

def F(v,t,k):
    result,err = quad(func=P,a=Vmin,b=v,args=(t,))  # args为函数P除自变量以外的参数t的值(元组)
    # result为积分值，err为算法的计算误差
    int_value = f(t,zeta=k) - result
    return int_value
Vmin

3.06832733

In [5]:
# 定义能量E函数(任意值)
def E(v,t,s=np.array([-799.28947485,-0.96842873,1.78234985,0.9679437])):
    dt = 1e-3 # 数值偏导数的微小增量
    e = F(v,t,k=s)-t*((F(v,t+dt,k=s)-F(v,t,k=s))/dt)
    return e*(6.241509326*0.001)  # 量纲换算

In [6]:
# 定义损失函数
def loss(theta):
    mse = 0
    for i in range(df_E.shape[0] - 1):
        e_pre = E(df_V[i],df_T[i],s=theta)
#         mse += (e_pre - df_E[i])**2
        if i < df_E.shape[0] - 2:
            mse += 0.9*(e_pre - df_E[i])**2
        else:
            mse += 0.1*(e_pre - df_E[i])**2
    return mse

E(df_V[0],df_T[0])

-8.062918548379997

In [7]:
# 优化
from scipy.optimize import minimize

# 初始化变量
theta0 = np.array([-799.28947485,-0.96842873,1.78234985,0.9679437])

result = minimize(loss,theta0,method ='Powell')
print(result)

   direc: array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])
     fun: 0.008191862598405064
 message: 'Optimization terminated successfully.'
    nfev: 122
     nit: 2
  status: 0
 success: True
       x: array([-8.01954293e+02,  7.95503295e-01,  1.78212020e+00,  9.67945102e-01])


In [8]:
# 绘制优化后的曲面E

grid_level = 40
C = result.x  # 优化后得到的参数
# C = np.array([-799.28947485,-0.96842873,1.78234985,0.9679437])

vv = np.linspace(Vmin,Vmax,grid_level)
tt = np.linspace(Tmin,Tmax,grid_level)
E_grid_pre = np.full((grid_level,grid_level),0.0)
error_grid = np.full((grid_level,grid_level),0.0)
vvv, ttt = np.meshgrid(vv, tt,indexing='ij')
vt_grid = np.c_[vvv.reshape((-1, 1)),  ttt.reshape((-1, 1))]
fig = plt.figure(figsize=(10, 8))
ax1 = fig.add_subplot(111, projection='3d')
for i in range(grid_level):
    for j in range(grid_level):
        E_grid_pre[i][j] = E(vt_grid[vv.shape[0]*i,0],vt_grid[j,1],s=C)

p1 = ax1.scatter(vt_grid[:,0], 
                vt_grid[:,1], 
                E_grid_pre, 
                alpha=0.3,
                linewidths=2,
                c='b'
#                 cmap='b'
                )
p2 = ax1.scatter(df_V,
                 df_T,
                 df_E,
                 alpha=1,
                 linewidths=2.5,
                 c='r')

ax1.set_xlabel(r'Volume ($\AA^{3}/atom$)', fontsize=15, labelpad=8)
ax1.set_ylabel('Temperature (K)', fontsize=15, labelpad=12)
ax1.set_zlabel('Eenergy (eV/atom)', fontsize=15, labelpad=9)
ax1.legend(['Grid Data','DFT-Data'], loc='upper right', fontsize=15)